## Step 0: Importing Libraries and Installation

In [1]:
!pip install networkx


In [3]:
import json
import networkx as nx

## Step 1: Load Dataset

In [4]:
with open("graph_data.json", "r") as f:
    graph_data = json.load(f)

nodes = graph_data["nodes"]
edges = graph_data["edges"]

len(nodes), len(edges)


(25, 26)

## Step 2: Build a knowledge graph

`NEIGHBOR, CONNECTED_TO, NEAR` are undirected (you can go both ways). <br>
`MONITORS` is directed, but for now we’ll just add it normally.

In [5]:
G = nx.Graph()  # undirected is fine for now

# Add nodes with attributes
for node in nodes:
    node_id = node["id"]
    G.add_node(node_id, **node)

# Add edges with relation as an attribute
for edge in edges:
    src = edge["from"]
    dst = edge["to"]
    rel = edge["relation"]
    G.add_edge(src, dst, relation=rel)

print("Graph built!")
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Graph built!
Number of nodes: 25
Number of edges: 26


## Step 2.1: Build label and id mapping

Here, we want to call things using their human names, like "Intersection 4", "Central Hospital". Whereas in JSON we used IDs like INT_4, RD_8, HOSP_1. So we'll map human names to IDs.

In [6]:
# Build label-to-id and id-to-label maps
label_to_id = {}
id_to_label = {}

for node in nodes:
    node_id = node["id"]
    label = node.get("label", node_id)
    label_to_id[label] = node_id
    id_to_label[node_id] = label

# Quick check
label_to_id.get("Intersection 4"), label_to_id.get("Central Hospital")

('INT_4', 'HOSP_1')

## Step 2.2: Testing, find neighbors of a node (by label)

In [7]:
def get_neighbors_by_label(label: str):
    node_id = label_to_id.get(label)
    if node_id is None:
        print(f"Unknown node label: {label}")
        return []
    neighbor_ids = list(G.neighbors(node_id))
    neighbor_labels = [id_to_label[n] for n in neighbor_ids]
    return neighbor_labels

print("Neighbors of Intersection 4:")
print(get_neighbors_by_label("Intersection 4"))

Neighbors of Intersection 4:
['Haram Road', 'Intersection 3', 'Intersection 5', 'Main Fire Station']


## Step 2.3: Testing find a simple path between two nodes

In [8]:
def find_path_by_label(source_label: str, target_label: str):
    src_id = label_to_id.get(source_label)
    tgt_id = label_to_id.get(target_label)

    if src_id is None or tgt_id is None:
        print(f"Unknown label(s): {source_label}, {target_label}")
        return []

    try:
        path_ids = nx.shortest_path(G, src_id, tgt_id)
        path_labels = [id_to_label[n] for n in path_ids]
        return path_labels
    except nx.NetworkXNoPath:
        print(f"No path between {source_label} and {target_label}")
        return []

print("Path from Intersection 4 to Central Hospital:")
print(find_path_by_label("Intersection 4", "Central Hospital"))

Path from Intersection 4 to Central Hospital:
['Intersection 4', 'Intersection 5', 'Intersection 6', 'Intersection 7', 'Intersection 8', 'Central Hospital']


In [9]:
print(find_path_by_label("Intersection 4", "Hospital A"))


Unknown label(s): Intersection 4, Hospital A
[]
